In [ ]:
import sys
from pathlib import Path

# In Jupyter notebooks, __file__ is not defined. Use Path.cwd().parent for PROJECT_ROOT.
PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))           
sys.path.append(str(PROJECT_ROOT / "src"))   

RANDOM_STATE = 42

print("PYTHONPATH patched:", sys.path[-2:]) 

In [ ]:
import pandas as pd
TARGET = "Survived" 
df_raw = pd.read_csv('../data/raw/Titanic-Dataset.csv')
X = df_raw.drop(columns=[TARGET])
y = df_raw[TARGET]

In [ ]:
# Split the dataset into training and testing sets with stratification
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE
)

In [ ]:
# Define numerical and categorical columns
num_cols = ["Age", "SibSp", "Parch", "Fare"]
cat_cols = ["Sex", "Pclass", "Embarked"]


In [ ]:
from src.preprocessing import build_preprocessing_hgb_native
# Build the preprocessing pipeline
preprocessing, cat_idx = build_preprocessing_hgb_native(num_cols, cat_cols)
Xt = preprocessing.fit_transform(X_train)


In [ ]:
from scipy.stats import randint, uniform  # Use scipy.stats for distributions
from sklearn.pipeline import Pipeline
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV


# Define the final HGB model with specified hyperparameters
hgb_final = HistGradientBoostingClassifier(
    learning_rate=0.05,
    max_iter=150,
    max_leaf_nodes=30,
    min_samples_leaf=21,
    categorical_features=cat_idx,
    random_state=RANDOM_STATE
)

# Update the pipeline with the final model
pipe_final = Pipeline([("preprocess", preprocessing), ("model", hgb_final)])

# Fit the pipeline with the training data
pipe_final.fit(X_train, y_train)

Base (RandomizedSearch best):
    learning_rate ≈ 0.0678,
    max_iter = 121,
    max_leaf_nodes = 39,
    min_samples_leaf = 21,
    CV AP ≈ 0.855

Local grid around best (for GridSearch):
    learning_rate: [0.05, 0.07, 0.09],
    max_leaf_nodes: [30, 39, 45],
    min_samples_leaf: [15, 21, 27],
    max_iter: fixed to 150

other HGB params: as in default or best from RandomizedSearch